# Facial Expression Detection using Keras 

# Importing packages and modules


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
config=tf.compat.v1.ConfigProto()
session=tf.compat.v1.Session(config=config)


import os

from keras.models import Sequential
from keras.layers import Conv2D , Dense , MaxPooling2D , Flatten , BatchNormalization
from keras import optimizers
from keras.preprocessing.image import load_img , img_to_array 
from keras.preprocessing.image import ImageDataGenerator


# Image Data Augmentation

In [2]:
# initally the data is unequaly distributed ,so this leads to biasnes towards a particular class.
# so i decided to do image data augmentation , i.e.. creating more data from exsisting data

# For the data augmentation , i choose to :

# 1. randomly rotate the image 
# 2. shifting the width and height of image 
# 3. flipping the image
# 4. randomly zooming the image

# Datagen1 will generate new image after processing the random operation

Datagen1 = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# expression represent of each class folder name like angry, sad , happy and so on.
for expression in os.listdir('./augmented_facial/images/train'):
        
        # current expreesion_images is a list of all images name in a paricular class folder 
        current_expression_images=os.listdir('./augmented_facial/images/train/'+expression)
        current_expression_size=len(current_expression_images)
        
        increasing_factor=0
        
        # deciding the increasing_factor for generating new images depending upon initial count.
        if current_expression_size < 3000:
            increasing_factor=12
        elif current_expression_size <4500:
            increasing_factor=1
        
        if increasing_factor :
               for images in current_expression_images:
                            # generating new images corrosponds to each image 
                            img = load_img('./augmented_facial/images/train/'+expression+'/'+images)
                            input_img = img_to_array(img)
                            input_img = input_img.reshape((1,)+input_img.shape)
                            temp = 1 
                            for batch in Datagen1.flow( input_img, batch_size=1,save_to_dir='./augmented_facial/images/train/'+
                                                       expression , save_format= "jpeg"):
                                if temp == increasing_factor:
                                    break
                                else :
                                    temp+=1

# Importing training and validation data 

In [6]:
# using .flow_from_directory() to upload images 
# parameters passing :
# 1. image path 
# 2. image size and colour 
# 3. batch size ( as it divides the data into batches and then pass a single batch a time to the model )
# 4. mode of classification 

batch_size = 64
Datagen2 = ImageDataGenerator()

training_data = Datagen2.flow_from_directory("./augmented_facial/images/train/",target_size=(48,48),
                                                                            color_mode='grayscale',
                                                                            batch_size=batch_size,
                                                                             class_mode='categorical',
                                                                             shuffle=True)

validation_data = Datagen2.flow_from_directory("./facial/images/validation/",target_size=(48,48),
                                                                            color_mode='grayscale',
                                                                            batch_size=batch_size,
                                                                             class_mode='categorical',
                                                                             shuffle=True)
# training ans testing data size 

print(training_data.n)
print(validation_data.n)

Found 42207 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
42207
7066


# Training the model

In [7]:
# Now we will make our Keras model. Our model will be a seqeuntial model 
# and generally all the time you can use seqeuntial model only

model=Sequential()

# My structure of the model looks something like this:

# (Conv2D()-->Conv2D()-->BatchNormalization()-->Maxpooling2D())*2--->Flatten()-->Dense()-->Dense()-->Dense()(output)

# first two cnn layer  followed by batchnormalization layer and a max pooling layer 
# using 64 units in first cnn layer and 128 units in second cnn layer

model.add(Conv2D( 64,(3,3),strides=(1,1) , padding="same",activation="relu", input_shape=(48,48,1),use_bias=True))
model.add(Conv2D(128,(3,3),strides=(1,1),padding="same",activation="relu",use_bias=True))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="same"))

# next two cnn layer  followed by batchnormalization layer and a max pooling layer 
# using 256 units in third cnn layer and 512 units in fourth cnn layer

model.add(Conv2D( 256,(3,3),strides=(1,1) , padding="same",activation="relu", use_bias=True))
model.add(Conv2D(512,(3,3),strides=(1,1),padding="same",activation="relu",use_bias=True))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="same"))

# flattening the multidimensional output of cnn layer into one dimensional array 
model.add(Flatten())

# using two dense layer in my neural network 
# first layers contains 256 units and next layer contains 512 units 
model.add(Dense(256,activation="relu",use_bias=True))
model.add(Dense(512,activation="relu",use_bias=True))

# output layer with 7 units (each unit corrosponds to one class) 
model.add(Dense(7,activation="softmax",use_bias=True))

# compiling my model
# using adam optimizer
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [8]:
# In order to make the optimizer converge faster and closest to the global minimum of the loss function, 
# i used an annealing method of the learning rate (LR).
# The higher LR, the bigger are the steps and the quicker is the convergence. 
# However the sampling is very poor with an high LR and the optimizer could probably fall into a local minima.
# Its better to have a decreasing learning rate during the training to reach 
# efficiently the global minimum of the loss function.
# To keep the advantage of the fast computation time with a high LR, 
# i decreased the LR dynamically every X steps (epochs) depending if it is necessary (when accuracy is not improved).
# With the ReduceLROnPlateau function from Keras.callbacks, 
# i choose to reduce the LR by 0.2 if the accuracy is not improved after 2 epochs.

Reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1,patience=2,min_lr=0.00001,model="auto")

In [9]:
# number of steps in a single epoch
# = len(input_data)/batch_size
steps_per_epoch = training_data.n // training_data.batch_size
validation_steps = validation_data.n // validation_data.batch_size
steps_per_epoch , validation_steps

(659, 110)

In [10]:
# using .fit_generator() to run my model
# using 10 epochs
# parameters : input data  and validation data 
#           : steps per epoch during training and validation steps for testing
#           : number of epochs
epoch = 10
model.fit_generator( training_data , steps_per_epoch = steps_per_epoch , epochs = epoch, 
                     validation_data = validation_data , validation_steps = validation_steps)


Epoch 1/10
659/659 [==============================] - 5398s 8s/step - loss: 2.1420 - accuracy: 0.2702 - val_loss: 2.0795 - val_accuracy: 0.2408
Epoch 2/10
659/659 [==============================] - 6366s 10s/step - loss: 1.5333 - accuracy: 0.3706 - val_loss: 1.5039 - val_accuracy: 0.4245
Epoch 3/10
659/659 [==============================] - 5972s 9s/step - loss: 1.3685 - accuracy: 0.4410 - val_loss: 2.6910 - val_accuracy: 0.3245
Epoch 4/10
659/659 [==============================] - 3970s 6s/step - loss: 1.2480 - accuracy: 0.4932 - val_loss: 1.2857 - val_accuracy: 0.5046
Epoch 5/10
659/659 [==============================] - 5118s 8s/step - loss: 1.1446 - accuracy: 0.5379 - val_loss: 1.4823 - val_accuracy: 0.5084
Epoch 6/10
659/659 [==============================] - 3852s 6s/step - loss: 1.0421 - accuracy: 0.5844 - val_loss: 1.1657 - val_accuracy: 0.5181
Epoch 7/10
659/659 [==============================] - 3809s 6s/step - loss: 0.9280 - accuracy: 0.6347 - val_loss: 1.3236 - val_accurac

# Converting Model to Json File 

In [11]:
# converting my model to json file

model_json = model.to_json()
with open("mode_json","w") as json_file:
    json_file.write(model_json)
    
# storing weights
model.save_weights("model_weights.h5")